In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
np.set_printoptions(precision=5, suppress=True)

In [ ]:
from quantum_masala.core import RealLattice, UPFv2Data, AtomBasis, Crystal
from quantum_masala.core.constants import angstrom, E_ryd, E_ryd_SI, eV

reallat = RealLattice.from_alat(5.1070, [0.5, 0.5, 0.5], [-0.5, 0.5, 0.5], [-0.5, -0.5, 0.5])

fe_oncv = UPFv2Data.from_file('fe', './fe/Fe_ONCV_PBE-1.2.upf')

fe_atoms = AtomBasis.from_cart(reallat, fe_oncv, 55.847, [0., 0., 0.])

crystal = Crystal(reallat, [fe_atoms])

In [ ]:
reallat = crystal.reallat
recilat = crystal.recilat

print(f"alat = {reallat.alat}")
print("Axes of real space in cartesian")
for i, ai in enumerate(reallat.axes_cart):
    print(f"a({i+1}) = {ai}")
print()
print("Axes of real space in alat")
for i, ai in enumerate(reallat.axes_alat):
    print(f"a({i+1}) = {ai}")
print('-'*40)
print()


print(f"tpiba = {recilat.tpiba}")
print("Axes of reciprocal space in tpiba")
for i, bi in enumerate(recilat.axes_tpiba):
    print(f"b({i+1}) = {bi}")
print()
print("Axes of reciprocal space in cart")
for i, bi in enumerate(recilat.axes_cart):
    print(f"b({i+1}) = {bi}")
print('-'*40)
print()

for sp in crystal.l_species:
    print(f"Label: {sp.label}")
    print(f"PseudoPot File: {sp.ppdata.filename}")
    print(f"numatoms:{sp.numatoms}")
    print(f"coords: \n{sp.cryst.T}")
    print()
print(f"numel: {crystal.numel}")
print('-'*40)
print()


In [ ]:
from quantum_masala.core import KPoints, GSpace

grho = GSpace(crystal.recilat, 80)
print(grho.grid_shape)
print(grho.numg)

In [ ]:
from quantum_masala.core import Rho

numspin = 2
rho = Rho(crystal, grho, numspin)

In [ ]:
from quantum_masala.core import KPoints

kpts = KPoints.mpgrid(crystal, [8, 8, 8], [False, False, False])
print(kpts.cryst.T, kpts.weights)

In [ ]:
kpts = KPoints.from_tpiba(recilat,
                          ([   0.0000000,   0.0000000,   0.0000000],  0.0019531),
                          ([   0.0000000,  -0.1250000,   0.1250000],  0.0234375),
                          ([   0.0000000,  -0.2500000,   0.2500000],  0.0234375),
                          ([   0.0000000,  -0.3750000,   0.3750000],  0.0234375),
                          ([   0.0000000,   0.5000000,  -0.5000000],  0.0117188),
                          ([  -0.1250000,  -0.1250000,   0.2500000],  0.0468750),
                          ([  -0.1250000,  -0.2500000,   0.3750000],  0.0937500),
                          ([  -0.1250000,   0.6250000,  -0.5000000],  0.0937500),
                          ([  -0.2500000,   0.7500000,  -0.5000000],  0.0468750),
                          ([  -0.2500000,   0.6250000,  -0.3750000],  0.0468750),
                          ([   0.0000000,   0.0000000,   0.2500000],  0.0117188),
                          ([   0.0000000,  -0.1250000,   0.3750000],  0.0468750),
                          ([   0.0000000,  -0.2500000,   0.5000000],  0.0468750),
                          ([   0.0000000,   0.6250000,  -0.3750000],  0.0234375),
                          ([  -0.1250000,  -0.1250000,   0.5000000],  0.0468750),
                          ([  -0.1250000,   0.7500000,  -0.3750000],  0.0937500),
                          ([  -0.2500000,   0.2500000,   0.2500000],  0.0156250),
                          ([  -0.2500000,   0.7500000,  -0.2500000],  0.0156250),
                          ([   0.6250000,  -0.6250000,   0.2500000],  0.0468750),
                          ([   0.5000000,  -0.5000000,   0.2500000],  0.0234375),
                          ([   0.5000000,  -0.6250000,   0.3750000],  0.0468750),
                          ([   0.0000000,   0.0000000,   0.5000000],  0.0117188),
                          ([   0.0000000,  -0.1250000,   0.6250000],  0.0468750),
                          ([   0.0000000,   0.7500000,  -0.2500000],  0.0234375),
                          ([  -0.1250000,   0.8750000,  -0.2500000],  0.0468750),
                          ([   0.5000000,  -0.5000000,   0.5000000],  0.0039062),
                          ([   0.0000000,   0.0000000,   0.7500000],  0.0117188),
                          ([   0.0000000,   0.8750000,  -0.1250000],  0.0234375),
                          ([   0.0000000,   0.0000000,  -1.0000000],  0.0019531)
                         )

print(kpts.cryst.T, kpts.weights)

In [ ]:
from quantum_masala.core import KPointsKgrp, WavefunK, Wfn2Rho

numbnd = 12

WfnK = WavefunK(rho.grho, numspin, numbnd)

kpts_kgrp = KPointsKgrp(kpts)
#print(kpts_kgrp.cryst)
l_wfnk = [WfnK(k_cryst, k_weight, ik) for ik, (k_cryst, k_weight) in enumerate(kpts_kgrp)]
for wfnk in l_wfnk:
    #print(wfnk.gwfc.numgk)
    wfnk.init_random_wfc()

rho_wfn = Wfn2Rho(crystal, WfnK)

In [ ]:
wfnk = l_wfnk[0]
print(wfnk.evl.shape)
print(wfnk.occ.shape)
print(wfnk.evc_gk.shape)

In [ ]:
from quantum_masala.pw.pot import PPDataLocal, PPDataNonLocal

l_pploc = [PPDataLocal(sp, rho.grho) for sp in crystal.l_species]
l_ppnl = [PPDataNonLocal(sp, WfnK.gspc) for sp in crystal.l_species]

In [ ]:
mag_start = [0.1, ]

if numspin == 2:
    rho_g = np.zeros((2, grho.numg), dtype="c16")
    for isp, pploc in enumerate(l_pploc):
        rho_g[0] += pploc.rhoatomic_g * (1 + mag_start[isp]) / 2
        rho_g[1] += pploc.rhoatomic_g * (1 - mag_start[isp]) / 2
else:
    rho_g = sum(pploc.rhoatomic_g for pploc in l_pploc).reshape(1, grho.numg)

rho.update(rho_g)

In [ ]:
rho_test = np.load('rho_atom_g.npy')
print(rho_test)
print(rho.g)
print(np.abs(rho.g - rho_test))

In [ ]:
from quantum_masala.pw.pot import Hartree, ExchCorr, Ionic
from quantum_masala.pw.pot import compute_ewald_en

vhart = Hartree(rho)
vxc = ExchCorr(rho, 'gga_x_pbe', 'gga_c_pbe')
vion = Ionic(rho, l_pploc)
e_ewald = compute_ewald_en(crystal, rho.grho)

print(f"Ewald Energy: {e_ewald*2} Ry")

In [ ]:
from quantum_masala.pw.occ import SmearMod
from quantum_masala.core.constants import eV, E_ryd

occ_mod = SmearMod(WfnK, crystal.numel, "gauss", 0.01*E_ryd) 
print(occ_mod.degauss)

In [ ]:
from quantum_masala.pw.ham import HamK
from quantum_masala.pw.eigsolve.david import david_solve

vbare_g0 = vion.g[0] / np.prod(vion.grho.grid_shape)

def compute_vloc_r():
    vhart.compute()
    vxc.compute()
    return vion.r + vhart.r + vxc.r

vloc_r = compute_vloc_r()
for ik, wfnk in enumerate(l_wfnk):
    #print(ik)
    hamk = HamK(wfnk, vloc_r, l_ppnl)
    for isp in range(wfnk.numspin):
        david_solve(wfnk, hamk, isp, 1E-2, vbare_g0)
    #print(wfnk.evl / eV)

In [ ]:
l_wfnk = occ_mod.compute(l_wfnk)
for ik, wfnk in enumerate(l_wfnk):
    print(ik)
    print(wfnk.occ)
    print(wfnk.evl)
rho_wfn.update(l_wfnk)

In [ ]:
from quantum_masala.pw.mix import GenBroyden

mix_mod = GenBroyden(rho, rho_wfn, 0.7, 8)

In [ ]:
e_err = mix_mod.compute_error(rho, rho_wfn)
print(e_err)

In [ ]:
from quantum_masala.core.constants import E_ryd

diago_thr = 1E-2
conv_thr = 1E-10
scf_converged = False

idxiter = 0
while not scf_converged:
    vloc_r = compute_vloc_r()
    for ik, wfnk in enumerate(l_wfnk):
        #print(ik)
        hamk = HamK(wfnk, vloc_r, l_ppnl)
        for isp in range(wfnk.numspin):
            david_solve(wfnk, hamk, isp, diago_thr, vbare_g0)
            
    l_wfnk = occ_mod.compute(l_wfnk)
    rho_wfn = rho_wfn.update(l_wfnk)
    
    idxiter += 1
    print(f"Iter #{idxiter}")
    e_err = mix_mod.compute_error(rho, rho_wfn)
    print(f"Diago Thr = {diago_thr}, Error = {e_err}")
    
    if e_err < conv_thr * E_ryd:
        scf_converged = True
    if idxiter == 1 and e_err < diago_thr * max(1.0, rho.numel):
        print("Large diago_thr")
        diago_thr = 0.1 * e_err / max(1.0, rho.numel)
    else:
        diago_thr = min(0.1 * e_err / max(1.0, rho.numel), diago_thr)
        diago_thr = max(diago_thr, 1e-13)
        rho = mix_mod.mix(rho, rho_wfn)
    
    print(f"Hart: {vhart.en / E_ryd} Ry")
    print(f"XC: {vxc.en / E_ryd} Ry")